In [1]:
class Value:
  def __init__(self, inData, inOperation = "", inChildren = None, inLabel = ""):
    self.data = float(inData)
    self.grad = 0.0
    self.operation = inOperation
    self.nBackward = lambda: None
    self.label = inLabel

    
    self.children = []
    if (inChildren is not None):
      self.children = [children for children in inChildren]
    
    
  def __repr__(self):
    return f"Value[{self.label}]({self.data}), {self.operation if self.operation != '' else 'none'}, ( {len(self.children)} ), grad = {self.grad}"

  # Operators

  def __add__(self, other):
    out = Value(self.data + other.data, "add", [self, other])

    def nBackward():
      self.grad += 1 * out.grad
      other.grad += 1 * out.grad

    out.nBackward = nBackward
    
    return out
  
  def __neg__(self):
    out = Value(- self.data, "neg", [self])

    def nBackward():
      self.grad += -1 * out.grad

    out.nBackward = nBackward
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, "mul", [self, other])
    
    def nBackward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out.nBackward = nBackward
    
    return out

  def __pow__(self, inPow):
    out = Value(self.data ** inPow, "pow", [self])

    def nBackward():
      self.grad += inPow * (self.data ** (inPow - 1)) * out.grad

    out.nBackward = nBackward
    
    return out

  def __truediv__ (self, other):
    out = Value(self.data / other.data, "div", [self, other])

    def nBackward():
      self.grad += 1/other.data * out.grad
      other.grad += self.data * -1 * (other.data**-2) * out.grad

    out.nBackward = nBackward
    
    return out


  def backward(self):
    graph = []
    visited = set()
  
    def buildPath(inNode):

      if (inNode not in visited):
        visited.add(inNode)
        for child in inNode.children:
          buildPath(child)
          
        graph.append(inNode)

    buildPath(self)
    self.grad = 1.0

    for it in reversed(graph):
      it.nBackward()

    def zeroGrad(self):
      def clear(inNode):
        inNode.grad = 0.0
        for child in inNode.children:
          zeroGrad(child)

      clear(self)

In [2]:
def printValues(inValue):
  
  valuesSet = set()

  def getValues(inValue):
    valuesSet.add(inValue)
    for value in inValue.children:
      getValues(value)

  getValues(inValue)
  for value in valuesSet:
    print(f"{value.data} - [{value.label}] - GRAD: {value.grad}")

In [3]:
a = Value(3.0, inLabel = "a")
b = Value(2.0, inLabel = "b")

c = a + b # 5
c.label = "c"

#d = 
c.backward()

print("#############")
print(c)
printValues(c)

#############
Value[c](5.0), add, ( 2 ), grad = 1.0
5.0 - [c] - GRAD: 1.0
3.0 - [a] - GRAD: 1.0
2.0 - [b] - GRAD: 1.0
